In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息-成长能力指标.xlsx')

In [3]:
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
0,1001,成长能力指标,2018-09-30,1271亿,--,432亿,430亿,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
1,1001,成长能力指标,2018-06-30,823亿,--,286亿,284亿,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
2,1001,成长能力指标,2018-03-31,396亿,--,143亿,143亿,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
3,1001,成长能力指标,2017-12-31,1686亿,--,543亿,540亿,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
4,1001,成长能力指标,2017-09-30,1255亿,--,419亿,413亿,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
5,1001,成长能力指标,2017-06-30,840亿,--,282亿,276亿,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
6,1001,成长能力指标,2017-03-31,429亿,--,145亿,140亿,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
7,1001,成长能力指标,2016-12-31,1608亿,--,531亿,517亿,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
8,1001,成长能力指标,2016-09-30,1209亿,--,407亿,397亿,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
9,1002,成长能力指标,2018-09-30,58.0亿,16.1亿,9.19亿,9.15亿,17.99%,-21.93%,-23.70%,5.20%,-11.87%,-13.78%


In [4]:
#将单位全部统一为以元为单位
def convert(cell):
    pattern1=(r'(-?\d+\.?\d+)亿')
    pattern2=(r'(-?\d+\.?\d+)万亿')
    pattern3=(r'(-?\d+\.?\d+)万')
    if re.findall(pattern1,cell)!=[]:
        cell=float(re.findall(pattern1, cell)[0])/10000
    elif re.findall(pattern2,cell)!=[]:
        cell=float(re.findall(pattern2, cell)[0])
    elif re.findall(pattern3,cell)!=[]:
        cell=float(re.findall(pattern3, cell)[0])/100000000
    else: cell=np.nan
    return cell

In [5]:
data['营业总收入(元)']=data['营业总收入(元)'].map(convert)

In [6]:
data['毛利润(元)']=data['毛利润(元)'].map(convert)

In [7]:
data['归属净利润(元)']=data['归属净利润(元)'].map(convert)

In [8]:
data['扣非净利润(元)']=data['扣非净利润(元)'].map(convert)

In [9]:
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
0,1001,成长能力指标,2018-09-30,0.127100,NaN,0.043200,0.043000,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
1,1001,成长能力指标,2018-06-30,0.082300,NaN,0.028600,0.028400,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
2,1001,成长能力指标,2018-03-31,0.039600,NaN,0.014300,0.014300,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
3,1001,成长能力指标,2017-12-31,0.168600,NaN,0.054300,0.054000,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
4,1001,成长能力指标,2017-09-30,0.125500,NaN,0.041900,0.041300,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
5,1001,成长能力指标,2017-06-30,0.084000,NaN,0.028200,0.027600,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
6,1001,成长能力指标,2017-03-31,0.042900,NaN,0.014500,0.014000,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
7,1001,成长能力指标,2016-12-31,0.160800,NaN,0.053100,0.051700,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
8,1001,成长能力指标,2016-09-30,0.120900,NaN,0.040700,0.039700,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
9,1002,成长能力指标,2018-09-30,0.005800,0.001610,0.000919,0.000915,17.99%,-21.93%,-23.70%,5.20%,-11.87%,-13.78%


In [10]:
#将缺失值'--'变为nan
data=data.replace('--',np.nan)

In [11]:
#查看出现较多的时间
data['日期'].value_counts()

2016-12-31    3208
2017-09-30    3204
2017-06-30    3195
2017-03-31    3178
2017-12-31    3133
2018-03-31    3104
2016-09-30    3092
2018-06-30    3065
2018-09-30    3031
2016-06-30     193
2016-03-31     142
2015-12-31     140
2015-09-30      71
2018-12-31      46
2014-12-31      31
2015-06-30       6
2013-12-31       5
15-06-30         4
15-09-30         4
16-03-31         4
15-03-31         4
15-12-31         4
16-06-30         4
14-12-31         3
16-12-31         2
14-09-30         2
14-06-30         2
2011-12-31       2
2015-03-31       2
16-09-30         2
2012-12-31       2
17-03-31         1
2010-12-31       1
Name: 日期, dtype: int64

In [12]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [13]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [14]:
#再次查看出现较多的时间
data['日期'].value_counts()

2016-12-31    2985
2017-09-30    2982
2017-12-31    2980
2018-06-30    2974
2018-09-30    2973
2017-06-30    2972
2018-03-31    2957
2017-03-31    2956
2016-09-30    2873
2016-06-30     156
2015-12-31     128
2016-03-31     112
2015-09-30      60
2018-12-31      45
2014-12-31      33
2015-06-30      10
2015-03-31       6
2013-12-31       5
2012-12-31       2
2014-06-30       2
2014-09-30       2
2011-12-31       2
2010-12-31       1
Name: 日期, dtype: int64

In [15]:
#将数量少于2800的日期全部丢弃
timelist=list(['2016-12-31','2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31','2017-03-31','2016-09-30'])
data=data[data['日期'].isin(timelist)]
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
0,1001,成长能力指标,2018-09-30,0.127100,NaN,0.043200,0.043000,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
1,1001,成长能力指标,2018-06-30,0.082300,NaN,0.028600,0.028400,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
2,1001,成长能力指标,2018-03-31,0.039600,NaN,0.014300,0.014300,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
3,1001,成长能力指标,2017-12-31,0.168600,NaN,0.054300,0.054000,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
4,1001,成长能力指标,2017-09-30,0.125500,NaN,0.041900,0.041300,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
5,1001,成长能力指标,2017-06-30,0.084000,NaN,0.028200,0.027600,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
6,1001,成长能力指标,2017-03-31,0.042900,NaN,0.014500,0.014000,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
7,1001,成长能力指标,2016-12-31,0.160800,NaN,0.053100,0.051700,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
8,1001,成长能力指标,2016-09-30,0.120900,NaN,0.040700,0.039700,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
9,1002,成长能力指标,2018-09-30,0.005800,0.001610,0.000919,0.000915,17.99%,-21.93%,-23.70%,5.20%,-11.87%,-13.78%


In [16]:
#转换数据格式
def get_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'[\d.]+',cell)
    if not _:
        return None
    return float(_[0])

In [17]:
data.iloc[:,3:]=data.iloc[:,3:].applymap(get_number)

In [18]:
data.sort_values(by=['企业编号','日期'],inplace=True)

In [19]:
#查看排序结果是否达到目标效果
data.columns=pd.Index(['企业编号', '标题', '日期', '$营业总收入(元)', '$毛利润(元)', '$归属净利润(元)', '$扣非净利润(元)',
       '营业总收入同比增长(元)', '归属净利润同比增长(元)', '扣非净利润同比增长(元)', '营业总收入滚动环比增长(元)',
       '归属净利润滚动环比增长(元)', '扣非净利润滚动环比增长(元)'])

In [20]:
data=data.set_index(['企业编号','日期'])

In [21]:
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题')
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息-成长能力指标.pickle')

In [22]:
data.index

MultiIndex(levels=[['标题', '$营业总收入(元)', '$毛利润(元)', '$归属净利润(元)', '$扣非净利润(元)', '营业总收入同比增长(元)', '归属净利润同比增长(元)', '扣非净利润同比增长(元)', '营业总收入滚动环比增长(元)', '归属净利润滚动环比增长(元)', '扣非净利润滚动环比增长(元)', '企业编号'], ['2016-09-30', '2016-12-31', '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30', '']],
           labels=[[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8]],
           names=[None, '日期'])